In [5]:
import pandas as pd
import tmap as tmap
from faerun import Faerun

from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from matplotlib.colors import ListedColormap
from rdkit.Chem.Descriptors import ExactMolWt

import pandas as pd
import numpy as np

In [9]:
#Train test data
train_test_path = "../../data_for_modeling/train_test_data/HDAC2_train_test_data_final.xlsx"
train_dataset = pd.read_excel(train_test_path, sheet_name='train_dataset')
test_dataset = pd.read_excel(train_test_path, sheet_name='test_dataset')
validation_dataset = pd.read_excel(train_test_path, sheet_name='validation_dataset')

#Add data belonging
belong_col_name = "Belong_to"
train_dataset[belong_col_name] = "Training data"
test_dataset[belong_col_name] = "Testing data"
validation_dataset[belong_col_name] = "Validation data"

#Train_test_dataset = combine of all three
all_dataset = pd.concat([train_dataset, validation_dataset, test_dataset], axis=0)

#Getting the classifier
all_data_path = "../../data_for_modeling/raw_data/HDAC2_raw_data.xlsx"
zgb_classifer_df = pd.read_excel(all_data_path, sheet_name='zbg_classifier_id')

In [10]:
print(len(train_dataset), len(test_dataset), len(validation_dataset), len(all_dataset))

2016 281 504 2801


In [11]:
all_dataset.head()

,CID,SMILES,AVG_IC50_nM,FINAL_LABEL,ZBG Classified,Belong_to
0,44138033,COC(=O)c1ccc(CO/N=C/c2ccc(/C=C/C(=O)NO)cc2)cc1,15800.0,Inactive,1,Training data
1,164616411,CC[C@H](NC(=O)C1CN(C)C1)c1ncc(-c2cc3ccccc3nc2O...,50000.0,Inactive,15,Training data
2,135357843,O=C(NO)c1ccc(Cn2nnnc2-c2cccs2)cc1,3698.0,Inactive,1,Training data
3,142506189,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(NCc3ccc(C(...,7173.0,Inactive,1,Training data
4,155537112,CN(C)c1ccc(C(=O)N(CC(=O)NCc2ccccc2)Cc2ccc(C(=O...,283.0,Inactive,4,Training data


In [12]:
zgb_dict = zgb_classifer_df.set_index('Code')['Name'].to_dict()
zgb_dict

{1: 'Acid hydroxamic',
 2: 'N-substituted Hydroxamic acid',
 3: 'Carboxylic acid',
 4: 'Benzamide',
 5: 'Alkyl ketone, Aryl ketone, Epoxyketone',
 6: 'Cyclic peptides',
 8: 'Trifluoromethylketone, Trifluoromethyloxadiazole',
 9: 'Thiol',
 10: 'Boronic acid.',
 11: 'Disulfide',
 12: 'Amino acid derivative',
 13: 'Tropolone',
 14: '3-hydroxypyridin-2-thione',
 15: 'Carboxamide',
 16: 'Benzoyl-hydrazide',
 17: 'Hydroxypyrimidine',
 18: 'Hydroxyureas',
 19: 'Chalcone',
 20: 'Aniline-benzamid',
 21: 'Others'}

In [13]:
#MACCS
from tqdm import tqdm

def tmap_maccs_fpts(data):
    Maccs_fpts = []
    count = 0
    with tqdm(total=len(data), desc='Progress') as pbar:
        for i in data:
            try:
                mol = Chem.MolFromSmiles(i)
            except:
                print("An exception occurred with " + str(count))
                continue
            fpts = MACCSkeys.GenMACCSKeys(mol)
            mfpts = np.array(fpts)
            mfpts = tmap.VectorUint(mfpts)
            Maccs_fpts.append(mfpts)
            count += 1
            pbar.update(1)  # Update the progress bar
    return np.array(Maccs_fpts)

#Morgan2
def tmap_morgan_fpts(data):
    Morgan_fpts = []
    count = 0
    with tqdm(total=len(data), desc='Progress') as pbar:
        for i in data:
            try:
                mol = Chem.MolFromSmiles(i)
            except:
                print("An exception occurred with " + str(count))
                continue
            fpts = AllChem.GetMorganFingerprintAsBitVect(mol, 2, 1024)
            mfpts = np.array(fpts)
            mfpts = tmap.VectorUint(mfpts)
            Morgan_fpts.append(mfpts)
            count += 1
            pbar.update(1)  # Update the progress bar
    return Morgan_fpts

In [14]:
fps = tmap_maccs_fpts(all_dataset["SMILES"])

Progress:   4%|▎         | 103/2801 [00:00<00:02, 1018.38it/s]

Progress: 100%|██████████| 2801/2801 [00:02<00:00, 1017.43it/s]


# Generate Tmap Layout

Generate labels

In [15]:
ic50 = []
labels = []
belong_to_groups = []
zbg_classifier_groups = []
active_inactive_groups = []
molecular_weight = []
chembl_url = "https://www.ebi.ac.uk/chembl/compound_report_card/"
pubchem_url = "https://pubchem.ncbi.nlm.nih.gov/compound/"

for i, row in all_dataset.iterrows():
    smiles = row['SMILES']
    mol = AllChem.MolFromSmiles(smiles)
    cid = str(row['CID'])
    if cid[-1].isalpha():
        cid = cid[:-1]
    
    labels.append(
            f'{smiles}__<a href="{pubchem_url+str(cid)}" target="_blank">{smiles}</a>__{smiles}'.replace(
                "'", ""
            )
        )
    ic50.append(row['AVG_IC50_nM'])
    # zbg labels groups
    zbg_classifier_groups.append(zgb_dict[row['ZBG Classified']])
    #Active inactive label groups
    active_inactive_groups.append(row['FINAL_LABEL'])
    #Belong to groups
    belong_to_groups.append(row[belong_col_name])
    #Molecular weight
    smiles_mw = ExactMolWt(mol)
    molecular_weight.append(smiles_mw)

In [16]:
print(len(belong_to_groups), len(ic50), len(zbg_classifier_groups), len(active_inactive_groups), len(labels))

2801 2801 2801 2801 2801


__Preprocessing the groups labels__

In [17]:
import scipy.stats as ss
# Create the labels and the integer encoded array for the groups,
# as they're categorical
zgb_labels_groups, zgb_groups = Faerun.create_categories(zbg_classifier_groups)
activity_labels_groups, activity_groups = Faerun.create_categories(active_inactive_groups)
belong_labels_groups, belong_groups = Faerun.create_categories(belong_to_groups)
#scale IC50
ic50_ranked = ss.rankdata(np.array(ic50) / max(ic50)) / len(ic50)
mw_ranked = ss.rankdata(np.array(molecular_weight) / max(molecular_weight)) / len(molecular_weight)

In [18]:
belong_labels_groups

[(0, 'Testing data'), (1, 'Training data'), (2, 'Validation data')]

In [19]:
bits = 1024
k = 300
enc = tmap.Minhash(bits)
lf = tmap.LSHForest(bits, 128)
lf.batch_add(enc.batch_from_binary_array(fps))
lf.index()
cfg = tmap.LayoutConfiguration()
cfg.k = k
cfg.sl_repeats = 2
cfg.mmm_repeats = 2
cfg.node_size = 2
x, y, s, t, _ = tmap.layout_from_lsh_forest(lf, config=cfg)

In [20]:
len(x)

2801

In [ ]:
faerun = Faerun(view="front", clear_color="#e3d8c3", coords=False)
custom_cmap = ListedColormap(
    ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", 
     "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf", 
     "#000080", "#fcec03"],
    name="custom",
)
faerun.add_scatter(
        "chembl",
        {"x": x, 
         "y": y, 
         "c": [zgb_groups, belong_groups, activity_groups, ic50_ranked, mw_ranked], 
         "labels": labels},
        colormap=[custom_cmap, custom_cmap, custom_cmap, "viridis", "viridis"],
        point_scale=4.5,
        max_point_size=10,
        has_legend=True,
        categorical=[True, True, True, False, False],
        shader="smoothCircle",
        legend_labels=[zgb_labels_groups, belong_labels_groups, activity_labels_groups],
        selected_labels=["SMILES", "PubChem CID", "Name"],
        series_title=["ZGB Classifier group", "Dataset label group", "Activity label group", "IC50 (nM)", "Molecular Weight"],
        max_legend_label=[None, None, None, str(round(max(ic50))), str(round(max(molecular_weight)))],
        min_legend_label=[None, None, None, str(round(min(ic50))), str(round(min(molecular_weight)))],
        title_index=2,
        legend_title=""
    )
faerun.add_tree(
    "pubchem_tree", {"from": s, "to": t}, point_helper="chembl", color="#222222"
)
# You may see a different Tmap from what us generated since the index of the data-points are randomly generated depending on the runtime environment.
# Howerver, the way your tmap branches connected will be similar to ours.
faerun.plot(file_name="../../results/tmap/Your_Tmap_HDAC2", template="smiles")